# 🇹🇷 Türkçe Metinlerde Tokenization ve Ön İşleme


İnsanlar için bir cümleyi okumak ve kelimelere ayırmak son derece kolay ve doğal bir süreçtir. Ancak bilgisayarlar için metin, başlangıçta sadece bir karakter dizisinden ibarettir:

```
"BugünDoğalDilİşlemedersibaşlıyor"
```

Bilgisayar, bu metnin nerede başlayıp nerede bittiğini, hangi kısmın kelime olduğunu veya hangi kısmın anlam taşıdığını kendiliğinden bilemez. Bu nedenle, metni önce daha küçük ve anlamlı parçalara ayırmamız gerekir. İşte bu işleme tokenization denir.

**🤖 Türkçe neden özellikle ilginç (ve biraz zor)?**

Türkçe, eklemeli (agglutinative) bir dildir. Yani bir kelimeye birçok ek eklenerek çok uzun ve karmaşık yapılar oluşturulabilir:

```
ev
evler
evleriniz
evlerinizden
evlerinizdenmiş
```
Hatta bazen tek bir kelime, bir cümle kadar anlam taşıyabilir:

```
Başlayamayacaklarımızdanmışsınız
```

İnsanlar için bu normaldir. Ancak bilgisayarlar için bu yapıların doğru şekilde analiz edilebilmesi, doğru bir tokenization süreci gerektirir.




Bu notebook’ta Türkçe metinleri tokenize etmek için **bazı ek kütüphaneler** kullanacağız. Aşağıdaki komutlar, gerekli kütüphaneleri kurmak için kullanılır:

**📦 Kurulan Kütüphaneler ve Amaçları**

**1️⃣ zemberek-python**

Zemberek, Türkçe doğal dil işleme için geliştirilmiş güçlü bir kütüphanedir. Türkçe’nin eklemeli yapısını anlayabilen özel araçlar içerir.

[Zemberek](https://github.com/ahmetaa/zemberek-nlp)

**2️⃣ antlr4-python3-runtime**

Bu kütüphane, Zemberek’in düzgün çalışabilmesi için gerekli olan bir bağımlılıktır. Zemberek, metni analiz ederken belirli dil kurallarını kullanır. Bu kuralların yorumlanabilmesi için ANTLR altyapısına ihtiyaç duyulur.

Biz bu kütüphaneyi doğrudan kullanmayacağız, ancak Zemberek’in çalışması için gereklidir.

**3️⃣ unicode_tr**

Bu kütüphane, Türkçe karakterlerle doğru şekilde çalışmamızı sağlar. Standart string işlemleri bazen Türkçe karakterleri doğru şekilde işleyemez. unicode_tr kütüphanesi, Türkçe’ye özel doğru karakter dönüşümlerini yapmamızı sağlar.

In [1]:
!pip install zemberek-python
!pip install antlr4-python3-runtime==4.9
!pip install unicode_tr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 MB 8.3 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141211 sha256=d7090b61d0acb96eb2d4151c93f5d8f68020693061601c6e2efbbd1c6d01893e
  Stored in directory: /root/.cache/pip/wheels/3e/92/b7/08c6a108fc5bf6370a7540d11bbe9befc99b7e045ac7558d49
Successfully built antlr4-python3-runtime
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-python3-runtime-4.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
omegaconf 2.3.0 requires antlr4-python3-runtime==4.9.*, but you have antlr4-pyt

Tokenization işlemini test etmek için örnek bir Türkçe metin (sample document) tanımlayalım:

In [2]:
sample_doc = """
İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.
Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?" videosu izlerken buldu.

Proje teslim tarihi: 10 Kasım 2026.
Başak, "Bu sefer erken başlayacağım." dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.

Kullandığı bazı teknolojiler:
Python, spaCy, Zemberek, TensorFlow, PyTorch ve tabii ki Stack Overflow.

İletişim bilgileri:
Web sitesi: www.basakbuluz.com
E-posta: basak.buluz@gmail.com
Hashtag: #yapayzeka #nlp #finalhaftası

Not: Yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.
"""


sample_doc

'\nİstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.\nAncak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube\'da "Python mu Java mı?" videosu izlerken buldu.\n\nProje teslim tarihi: 10 Kasım 2026.\nBaşak, "Bu sefer erken başlayacağım." dedi, fakat GitHub\'a ilk commit\'i 09 Kasım 2026 saat 23:59\'da attı.\n\nKullandığı bazı teknolojiler:\nPython, spaCy, Zemberek, TensorFlow, PyTorch ve tabii ki Stack Overflow.\n\nİletişim bilgileri:\nWeb sitesi: www.basakbuluz.com\nE-posta: basak.buluz@gmail.com\nHashtag: #yapayzeka #nlp #finalhaftası\n\nNot: Yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.\n'

**📦 NLTK kütüphanesi**

NLTK, Doğal Dil İşleme alanında en eski ve en yaygın kullanılan kütüphanelerden biridir. Özellikle eğitim amaçlı kullanımı oldukça yaygındır.

**📥 Punkt tokenizer Modeli**

Punkt, cümle sınırlarını tespit etmek için kullanılan önceden eğitilmiş bir modeldir.

**🔧 Tokenization fonksiyonları**

NLTK’nin en sık kullanılan iki tokenization fonksiyonunu içe aktarıyoruz:

* word_tokenize → kelimelere ayırma
* sent_tokenize → cümlelere ayırma

[NLTK](https://www.nltk.org/)

In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


#### **sent_tokenize**

* sample_doc → tokenize etmek istediğimiz metindir
* sent_tokenize() → metni cümlelere ayıran NLTK fonksiyonudur
* language="turkish" → metnin Türkçe olduğunu belirtir

Bu işlem sonucunda, metin artık bir cümle listesi haline gelir

In [4]:
sentences = sent_tokenize(sample_doc, language="turkish")

for sentence in sentences:
  print(sentence)
  print("\n")


İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.


Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?"


videosu izlerken buldu.


Proje teslim tarihi: 10 Kasım 2026.


Başak, "Bu sefer erken başlayacağım."


dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.


Kullandığı bazı teknolojiler:
Python, spaCy, Zemberek, TensorFlow, PyTorch ve tabii ki Stack Overflow.


İletişim bilgileri:
Web sitesi: www.basakbuluz.com
E-posta: basak.buluz@gmail.com
Hashtag: #yapayzeka #nlp #finalhaftası

Not: Yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.




#### **word_tokenize**

Metni sadece cümlelere ayırmakla kalmıyor, aynı zamanda her bir cümleyi kelimelere (token’lara) da ayırıyoruz. Bu işleme **word tokenization (kelime seviyesinde tokenization)** denir.

In [5]:
for sentence in sentences:
  print(sentence)
  words = word_tokenize(sentence)
  print(words)
  print("\n")


İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.
['İstanbul', 'Gedik', 'Üniversitesi', 'Bilgisayar', 'Mühendisliği', 'öğrencisi', 'Başak', ',', 'NLP', 'ödevine', 'başlamak', 'için', 'bilgisayarını', 'açtı', '.']


Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?"
['Ancak', 'başlamadan', 'önce', 'sadece', '5', 'dakika', 'dinlenmek', 'istedi', 've', 'kendini', '2', 'saat', 'sonra', "YouTube'da", '``', 'Python', 'mu', 'Java', 'mı', '?', "''"]


videosu izlerken buldu.
['videosu', 'izlerken', 'buldu', '.']


Proje teslim tarihi: 10 Kasım 2026.
['Proje', 'teslim', 'tarihi', ':', '10', 'Kasım', '2026', '.']


Başak, "Bu sefer erken başlayacağım."
['Başak', ',', '``', 'Bu', 'sefer', 'erken', 'başlayacağım', '.', "''"]


dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.
['dedi', ',', 'fakat', 'GitHub', "'", 'a', 'ilk', 'commit', "'", 'i', '09', 'Kası

## Zemberek

**🇹🇷 TurkishSentenceExtractor nedir?**

TurkishSentenceExtractor, Türkçe metinleri cümlelere ayırmak (sentence tokenization) için geliştirilmiş bir araçtır.

Daha önce NLTK ile de cümlelere ayırma işlemi yaptık. Ancak Zemberek, özellikle Türkçe için geliştirildiği için bazı durumlarda daha doğru sonuçlar verebilir.

Çünkü Türkçe:

Eklemeli bir dildir
Özel karakterler içerir (ç, ğ, ı, ö, ş, ü)
Kesme işareti ile ayrılan ekler içerir ("Türkiye'nin", "Ankara'da")

Zemberek bu yapıları daha iyi anlayacak şekilde tasarlanmıştır.

**🎯 Neden Zemberek kullanıyoruz?**

Genel NLP kütüphaneleri (NLTK, spaCy vb.) birçok dil için çalışır, ancak Türkçe’ye özel değildir.

Zemberek ise:

Türkçe için özel olarak geliştirilmiştir
Türkçe dil kurallarını bilir
Türkçe metinleri daha doğru analiz edebilir

Bu nedenle, Türkçe NLP uygulamalarında Zemberek oldukça güçlü bir araçtır.

In [6]:
from zemberek import TurkishSentenceExtractor

#### zemberek - sentence tokenizer

In [7]:
extractor = TurkishSentenceExtractor()
sentences = extractor.from_paragraph(sample_doc)

for sentence in sentences:
  print(sentence)
  print("\n")

İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.


Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?


" videosu izlerken buldu.


Proje teslim tarihi: 10 Kasım 2026.
Başak, "Bu sefer erken başlayacağım.


" dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.


Kullandığı bazı teknolojiler:
Python, spaCy, Zemberek, TensorFlow, PyTorch ve tabii ki Stack Overflow.

İletişim bilgileri:
Web sitesi: www.basakbuluz.com
E-posta: basak.buluz@gmail.com
Hashtag: #yapayzeka #nlp #finalhaftası

Not: Yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.




#### zemberek - word tokenizer

In [8]:
from zemberek import TurkishTokenizer
from zemberek.tokenization.token import Token

tokenizer = TurkishTokenizer.builder().accept_all().ignore_types(
    [Token.Type.NewLine,
     Token.Type.SpaceTab,
     Token.Type.Punctuation]).build()


for sentence in sentences:
  print(sentence)
  tokens = tokenizer.tokenize(sentence)
  words = [token.content for token in tokens]
  print(words)
  print("\n")


İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.
['İstanbul', 'Gedik', 'Üniversitesi', 'Bilgisayar', 'Mühendisliği', 'öğrencisi', 'Başak', 'NLP', 'ödevine', 'başlamak', 'için', 'bilgisayarını', 'açtı']


Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?
['Ancak', 'başlamadan', 'önce', 'sadece', '5', 'dakika', 'dinlenmek', 'istedi', 've', 'kendini', '2', 'saat', 'sonra', "YouTube'da", 'Python', 'mu', 'Java', 'mı']


" videosu izlerken buldu.
['videosu', 'izlerken', 'buldu']


Proje teslim tarihi: 10 Kasım 2026.
Başak, "Bu sefer erken başlayacağım.
['Proje', 'teslim', 'tarihi', '10', 'Kasım', '2026.', 'Başak', 'Bu', 'sefer', 'erken', 'başlayacağım']


" dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.
['dedi', 'fakat', "GitHub'a", 'ilk', "commit'i", '09', 'Kasım', '2026', 'saat', "23:59'da", 'attı']


Kullandığı bazı teknolojiler:
Python, spaCy,

In [9]:
# TOKENIZATION
tokenizer = TurkishTokenizer.DEFAULT

tokens = tokenizer.tokenize("Saat 12:00.")
for token in tokens:
    print('Content = ', token.content)
    print('Type = ', token.type_.name)
    print('Start = ', token.start)
    print('Stop = ', token.end, '\n')

Content =  Saat
Type =  Word
Start =  0
Stop =  3 

Content =  12:00
Type =  Time
Start =  5
Stop =  9 

Content =  .
Type =  Punctuation
Start =  10
Stop =  10 



**🎯 Bu neden çok önemli?**

Çünkü artık sadece kelimeleri değil, aynı zamanda: **token türünü, metin içindeki konumunu ve yapısını** da analiz edebiliyoruz.

Bu bilgiler şu uygulamalarda kullanılır:

* Named Entity Recognition (NER)
* Bilgi çıkarımı
* Metin hizalama (alignment)
* Metin işleme pipeline’ları

### lower(): Dökümanı küçük harfe çevirir

**🎯 Neden lowercase işlemi yapıyoruz?**

Çünkü bilgisayar için:
```
Python
python
PYTHON
```
üçü farklı kelimeler olarak algılanır.
Ama çoğu NLP uygulamasında bunların aynı kelime olduğunu varsaymak isteriz.
Lowercase işlemi sayesinde:
```
Python → python
PYTHON → python
```
Hepsi aynı forma dönüşür.

Bu da:

Kelime sayımını kolaylaştırır
Veri tutarlılığını artırır
Model performansını iyileştirir

**⚠️ Türkçe için önemli bir detay**

Türkçe’de bazı harflerin dönüşümü İngilizce’den farklıdır:
```
I → ı
İ → i
```
Python’ın .lower() fonksiyonu genellikle doğru çalışır, ancak bazı özel durumlarda Türkçe’ye özel kütüphaneler (örneğin **unicode_tr**) daha doğru sonuç verebilir.



In [10]:
# -*- coding: utf-8 -*-
from unicode_tr import unicode_tr

In [11]:
sentences_lower = [sentence.lower() for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

i̇stanbul gedik üniversitesi bilgisayar mühendisliği öğrencisi başak, nlp ödevine başlamak için bilgisayarını açtı.


ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra youtube'da "python mu java mı?


" videosu izlerken buldu.


proje teslim tarihi: 10 kasım 2026.
başak, "bu sefer erken başlayacağım.


" dedi, fakat github'a ilk commit'i 09 kasım 2026 saat 23:59'da attı.


kullandığı bazı teknolojiler:
python, spacy, zemberek, tensorflow, pytorch ve tabii ki stack overflow.

i̇letişim bilgileri:
web sitesi: www.basakbuluz.com
e-posta: basak.buluz@gmail.com
hashtag: #yapayzeka #nlp #finalhaftası

not: yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.




In [12]:
sentences_lower = [unicode_tr(sentence).lower() for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

istanbul gedik üniversitesi bilgisayar mühendisliği öğrencisi başak, nlp ödevine başlamak için bilgisayarını açtı.


ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra youtube'da "python mu java mı?


" videosu izlerken buldu.


proje teslim tarihi: 10 kasım 2026.
başak, "bu sefer erken başlayacağım.


" dedi, fakat github'a ilk commit'i 09 kasım 2026 saat 23:59'da attı.


kullandığı bazı teknolojiler:
python, spacy, zemberek, tensorflow, pytorch ve tabii ki stack overflow.

iletişim bilgileri:
web sitesi: www.basakbuluz.com
e-posta: basak.buluz@gmail.com
hashtag: #yapayzeka #nlp #finalhaftası

not: yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.




**### clean_word():**

Bu adımda, metin içindeki kelimeleri temizlemek (cleaning) için kendi fonksiyonumuzu tanımlıyoruz. Bu işlem, NLP’de en önemli ön işleme (pre-processing) adımlarından biridir.

**🧹 clean_word() fonksiyonu ne yapıyor?**

Bu fonksiyonun amacı, bir kelimeyi NLP analizine uygun hale getirmek için temizlemek ve normalize etmektir.

Gerçek dünya metinleri genellikle şu tür "gürültüler" içerir:

```
"Ankara'da"
"Python!!!"
"%100"
"e-posta:"
"#yapayzeka"
```

Bu karakterler çoğu zaman analiz için gereksizdir.
Bu fonksiyon, kelimeyi sade ve analiz edilebilir hale getirir.

**🔹 Adım adım ne oluyor?**

1️⃣ Türkçe’ye uygun şekilde küçük harfe çevirme

2️⃣ Özel karakterleri temizleme

3️⃣ Türkçe alfabesi dışındaki karakterleri kaldırma



In [13]:
import re
import string

def clean_word(word):
  unwanted_list = [u"&bull;", u"&lsquo;",u",",u"?",u"!",u'"',u"'",u"‘",u"’",u"/",u"<",u">",u"|",u"“",";","&","(",")","=","+","-","\\","*",":","~","@","."]
  alpha_list = [u"a",u"b",u"c",u"ç",u"d",u"e",u"f",u"g",u"ğ",u"h",u"ı",u"i",u"j",u"k",u"l",u"m",u"n",u"o",u"ö",u"p",u"q",u"r",u"s",u"ş",u"t",u"u",u"ü",u"v",u"w",u"x",u"y",u"z"," ",]

  # küçük harfe çeviriliyor
  word = unicode_tr(word).lower()
  txt = word.replace(u"â", u"a").lower()  # word.lower()

  # farklı karakterler siliniyor
  for unwanted_char in unwanted_list:
    # txt = txt.replace(uw,' ')
    # "Ankara'da" gibi tırnak kaldırıldığında "Ankara da" şeklinde oluşan aradaki boşluk birleştirildi.
    txt = txt.replace(unwanted_char, "")

  # geçerli harf listesi dışındaki harfler siliniyor
  chars = list(set(txt))
  for char in chars:
    if not char in alpha_list:
      txt = txt.replace(char, "")

  return txt

In [14]:
clean_word("#yâpayzeka")

'yapayzeka'

In [15]:
for sentence in sentences:
  print(sentence)
  tokens = tokenizer.tokenize(sentence)
  for token in tokens:
    print(clean_word(token.content))
  print("\n")

İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.
istanbul
gedik
üniversitesi
bilgisayar
mühendisliği
öğrencisi
başak

nlp
ödevine
başlamak
için
bilgisayarını
açtı



Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?
ancak
başlamadan
önce
sadece

dakika
dinlenmek
istedi
ve
kendini

saat
sonra
youtubeda

python
mu
java
mı



" videosu izlerken buldu.

videosu
izlerken
buldu



Proje teslim tarihi: 10 Kasım 2026.
Başak, "Bu sefer erken başlayacağım.
proje
teslim
tarihi


kasım

başak


bu
sefer
erken
başlayacağım



" dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.

dedi

fakat
githuba
ilk
commiti

kasım

saat
da
attı



Kullandığı bazı teknolojiler:
Python, spaCy, Zemberek, TensorFlow, PyTorch ve tabii ki Stack Overflow.

İletişim bilgileri:
Web sitesi: www.basakbuluz.com
E-posta: basak.buluz@gmail.com
Hashtag: #yapayzeka #nlp #finalhaftası

No

## **remove_URL: Dökümandan URL'leri kaldırın**

Bu adımda, metin içindeki URL’leri (web adreslerini) temizlemek için özel bir fonksiyon tanımlıyoruz. URL’ler genellikle NLP analizinde gereksiz gürültü oluşturur ve çoğu durumda metinden kaldırılmaları gerekir.

In [17]:
def remove_URL(text):
  # Remove URLs from a sample string
  pattern_url = r"""(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
  text_clean = re.sub(pattern_url, "", text)

  return text_clean

In [18]:
sentences_lower = [remove_URL(sentence) for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.


Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?


" videosu izlerken buldu.


Proje teslim tarihi: 10 Kasım 2026.
Başak, "Bu sefer erken başlayacağım.


" dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.


Kullandığı bazı teknolojiler:
Python, spaCy, Zemberek, TensorFlow, PyTorch ve tabii ki Stack Overflow.

İletişim bilgileri:
Web sitesi: 
E-posta: basak.buluz@gmail.com
Hashtag: #yapayzeka #nlp #finalhaftası

Not: Yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.




## **remove_email: Dökümanlardan e-posta adreslerini temizleyin**

Bu fonksiyonun amacı, metin içindeki tüm e-posta adreslerini tespit etmek ve silmektir.

Örneğin şu tür yapıları kaldırabilir:
```
basak.buluz@gmail.com
student123@std.gedik.edu.tr
nlp.odev@universite.edu
```



In [19]:
def remove_email(text):
  pattern_email = "\S*@\S*\s?"
  text_clean = re.sub(pattern_email, "", text)
  return text_clean


<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-1658067703.py:2: SyntaxWarning: invalid escape sequence '\S'
  pattern_email = "\S*@\S*\s?"


In [20]:
sentences_lower = [remove_email(sentence) for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.


Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?


" videosu izlerken buldu.


Proje teslim tarihi: 10 Kasım 2026.
Başak, "Bu sefer erken başlayacağım.


" dedi, fakat GitHub'a ilk commit'i 09 Kasım 2026 saat 23:59'da attı.


Kullandığı bazı teknolojiler:
Python, spaCy, Zemberek, TensorFlow, PyTorch ve tabii ki Stack Overflow.

İletişim bilgileri:
Web sitesi: www.basakbuluz.com
E-posta: Hashtag: #yapayzeka #nlp #finalhaftası

Not: Yaklaşık ~12 saatlik uykusuzluk sonrası yazılan kodların çalışması tamamen şans eseridir.




## **NLTK - STOP WORDS**

Bu adımda, Türkçe metinlerde çok sık geçen ancak genellikle analiz açısından fazla anlam taşımayan kelimeleri (stop words) kaldırabilmek için gerekli listeyi yüklüyoruz.

**🧩 Stop word nedir?**

Stop words, bir dilde çok sık kullanılan ancak genellikle metnin ana anlamına büyük katkı sağlamayan kelimelerdir.

[trstop](https://github.com/ahmetax/trstop)

In [21]:
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_list = set(stopwords.words('turkish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
stopwords_list = stopwords.words('turkish')

# Tüm stop wordleri listele
for word in stopwords_list:
    print(word)

acaba
ama
aslında
az
bazı
belki
biri
birkaç
birşey
biz
bu
çok
çünkü
da
daha
de
defa
diye
eğer
en
gibi
hem
hep
hepsi
her
hiç
için
ile
ise
kez
ki
kim
mı
mu
mü
nasıl
ne
neden
nerde
nerede
nereye
niçin
niye
o
sanki
şey
siz
şu
tüm
ve
veya
ya
yani


In [22]:
for sentence in sentences:
  print(sentence)
  tokens = tokenizer.tokenize(sentence)
  words = [token.content for token in tokens]
  print(words)

  print("words_filtered")
  words_filtered = []
  for w in words:
    if w not in stopwords_list:
      words_filtered.append(w)

  print(words_filtered)
  print("\n")


İstanbul Gedik Üniversitesi Bilgisayar Mühendisliği öğrencisi Başak, NLP ödevine başlamak için bilgisayarını açtı.
['İstanbul', 'Gedik', 'Üniversitesi', 'Bilgisayar', 'Mühendisliği', 'öğrencisi', 'Başak', ',', 'NLP', 'ödevine', 'başlamak', 'için', 'bilgisayarını', 'açtı', '.']
words_filtered
['İstanbul', 'Gedik', 'Üniversitesi', 'Bilgisayar', 'Mühendisliği', 'öğrencisi', 'Başak', ',', 'NLP', 'ödevine', 'başlamak', 'bilgisayarını', 'açtı', '.']


Ancak başlamadan önce sadece 5 dakika dinlenmek istedi ve kendini 2 saat sonra YouTube'da "Python mu Java mı?
['Ancak', 'başlamadan', 'önce', 'sadece', '5', 'dakika', 'dinlenmek', 'istedi', 've', 'kendini', '2', 'saat', 'sonra', "YouTube'da", '"', 'Python', 'mu', 'Java', 'mı', '?']
words_filtered
['Ancak', 'başlamadan', 'önce', 'sadece', '5', 'dakika', 'dinlenmek', 'istedi', 'kendini', '2', 'saat', 'sonra', "YouTube'da", '"', 'Python', 'Java', '?']


" videosu izlerken buldu.
['"', 'videosu', 'izlerken', 'buldu', '.']
words_filtered
['"', 'vide